# Narupa Servers

In this notebook, we explore what goes into a Narupa server, and how to choose a flavour for a given application.

This gets into some of the nuts and bolts of how Narupa works under the hood, including some GRPC details. While understanding GRPC is not necessary, it [will help](grpc.io).

## The Server Stack

The Narupa libraries have a lot of servers lying around:

In [1]:
from narupa.app import NarupaApplicationServer, NarupaFrameApplication, NarupaImdApplication
from narupa.core import NarupaServer, GrpcServer
from narupa.trajectory import FrameServer
from narupa.imd import ImdServer
from narupa.multiplayer import MultiplayerServer

What do these all do? What should we be using?

Let's work our way from the bottom of the stack to the top

### GRPC Server

It all starts with the Grpc Server, which forms the base of all other servers.

In [2]:
grpc_server = GrpcServer(address='localhost', port=0)

By itself, this server doesn't do much other than set up the underlying [gRPC server](grpc.io) with a few little helpers:

In [3]:
print(grpc_server.__doc__)


    A base class for running GRPC servers that handles the starting and closing
    of the underlying server.

    :param address: The IP address at which to run the server.
    :param port: The port on which to run the server.
    


In [4]:
[x for x in dir(grpc_server) if not x.startswith('_')]

['add_service',
 'address',
 'address_and_port',
 'close',
 'logger',
 'port',
 'server',
 'setup_services']

Mainly, it provides a method to gracefully `close`, access to the address and port the server is running on, and the ability to set up new gRPC servers. 

If you just want to run a GRPC server with a couple of little helpers, this is the one for you.

In practice, Narupa servers always want commands and state synchronisation, so let's add those. 

In [5]:
from narupa.command import CommandService
from narupa.state.state_service import StateService

command_service = CommandService()
state_service = StateService()

These are the python implementations of the Command and State service. If we add them to the server, it will gain the ability to run commands and synchronise state (see the [commands and state](./commands_and_state.ipynb) notebook).

In [6]:
grpc_server.add_service(command_service)
grpc_server.add_service(state_service)

In [7]:
def hello():
    print('hi!')

command_service.register_command('hello', hello)

Let's check that worked

In [8]:
from narupa.core import NarupaClient

with NarupaClient.insecure_channel(address=grpc_server.address, port=grpc_server.port) as client:
    client.run_command('hello')

hi!


Cool! We've just created a functioning Narupa server! If you wanted to write your own GRPC services, you could add them with the same methodology. 

### Narupa Server

Since we almost always want commands and state synchronisation, we've created the Narupa Server object that does exactly that. 

In [9]:
narupa_server = NarupaServer(address='localhost', port=0)

In [13]:
print(narupa_server.__doc__)


    A base for Narupa gRPC servers. Sets up a gRPC server, and automatically
    attaches a :class:`CommandService`, enabling the running of arbitrary commands.
    


In [10]:
narupa_server.register_command('narupa_hello', hello)

In [11]:
with NarupaClient.insecure_channel(address=narupa_server.address, port=narupa_server.port) as client:
    client.run_command('narupa_hello')

hi!


That's the Narupa server, it's just a GRPC server with the command and state service added. 